In [ ]:
# | default_exp production.catalog

# production.catalog

> this submodule stores the main functions to create a catalog from the planet4 data.

In [ ]:
# | export

# p4tools package imports
import p4tools.catalog.io as io
import p4tools.catalog.metadata as p4meta
from p4tools.catalog.projection import X2LATLON, P4Mosaic, TileCaclulator, create_RED45_mosaic

# other imports
from tqdm import tqdm
import pandas as pd
import logging
import itertools
from planetarypy.pds.apps import get_index

#typing imports
from collections.abc import Iterable
from logging import Logger
from pandas import DataFrame

In [ ]:
# | export 

#starting the logger
LOGGER: Logger = logging.getLogger(__name__)

In [ ]:
# | export
class ReleaseManager:
    """Class to manage releases and find relevant files.
    TODO better description
    Parameters
    ----------
    version : str
        Version string for this catalog. Same as datapath in other P4 code.
    obsids : iterable, optional
        Iterable of obsids that should be used for catalog file. Default is to use the full list of the default database, which is Seasons 2 and 3 at this point.
    overwrite : bool, optional
        Switch to control if already existing result folders for an obsid should be overwritten.
        Default: False
    """

    DROP_FOR_TILE_COORDS: list[str] = [
        "xy_hirise",
        "SampleResolution",
        "LineResolution",
        "PositiveWest360Longitude",
        "Line",
        "Sample",
    ]

    FAN_COLUMNS_AS_PUBLISHED: list[str] = [
        "marking_id",
        "angle",
        "distance",
        "tile_id",
        "image_x",
        "image_y",
        "n_votes",
        "obsid",
        "spread",
        "version",
        "vote_ratio",
        "x",
        "y",
        "x_angle",
        "y_angle",
        "l_s",
        "map_scale",
        "north_azimuth",
        "BodyFixedCoordinateX",
        "BodyFixedCoordinateY",
        "BodyFixedCoordinateZ",
        "PlanetocentricLatitude",
        "PlanetographicLatitude",
        "Longitude",
    ]
    BLOTCH_COLUMNS_AS_PUBLISHED: list[str] = [
        "marking_id",
        "angle",
        "tile_id",
        "image_x",
        "image_y",
        "n_votes",
        "obsid",
        "radius_1",
        "radius_2",
        "vote_ratio",
        "x",
        "y",
        "x_angle",
        "y_angle",
        "l_s",
        "map_scale",
        "north_azimuth",
        "BodyFixedCoordinateX",
        "BodyFixedCoordinateY",
        "BodyFixedCoordinateZ",
        "PlanetocentricLatitude",
        "PlanetographicLatitude",
        "Longitude",
    ]

    def __init__(self, version, obsids=None, overwrite=False, dbname=None):
        self.catalog = f"P4_catalog_{version}"
        self.overwrite = overwrite
        self._obsids: Iterable | None = obsids
        self.dbname = dbname

    @property
    def savefolder(self):
        "Path to catalog folder"
        return io.data_root / self.catalog

    @property
    def metadata_path(self):
        "Path to catalog metadata file."
        return self.savefolder / f"{self.catalog}_metadata.csv"

    @property
    def tile_coords_path(self):
        "Path to catalog tile coordinates file."
        return self.savefolder / f"{self.catalog}_tile_coords.csv"

    @property
    def tile_coords_path_final(self):
        "Path to final catalog tile coordinates file."
        return self.savefolder / f"{self.catalog}_tile_coords_final.csv"

    @property
    def obsids(self):
        """Return list of obsids for catalog production.

        If ._obsids is None, get default full obsids list for current default P4 database.
        """
        if self._obsids is None:
            db = io.DBManager(dbname=self.dbname)
            self._obsids = db.obsids
        return self._obsids

    @obsids.setter
    def obsids(self, values):
        self._obsids = values

    @property
    def fan_file(self):
        "Return path to fan catalog file."
        try:
            return next(self.savefolder.glob("*_fan.csv"))
        except StopIteration:
            print(f"No file found. Looking at {self.savefolder}.")

    @property
    def blotch_file(self):
        "Return path to blotch catalog file."
        try:
            return next(self.savefolder.glob("*_blotch.csv"))
        except StopIteration:
            print(f"No file found. Looking at {self.savefolder}.")

    @property
    def fan_merged(self):
        return self.fan_file.parent / f"{self.fan_file.stem}_meta_merged.csv"

    @property
    def blotch_merged(self):
        return self.blotch_file.parent / f"{self.blotch_file.stem}_meta_merged.csv"

    def read_fan_file(self):
        return pd.read_csv(self.fan_merged)

    def read_blotch_file(self):
        return pd.read_csv(self.blotch_merged)

    def check_for_todo(self, overwrite=None):
        if overwrite is None:
            overwrite = self.overwrite
        bucket = []
        for obsid in self.obsids:
            pm = io.PathManager(obsid=obsid, datapath=self.savefolder)
            path = pm.obsid_results_savefolder / obsid
            if path.exists() and overwrite is False:
                continue
            else:
                bucket.append(obsid)
        self.todo = bucket

    def get_parallel_args(self):
        return [(i, self.catalog, self.dbname) for i in self.todo]

    def get_no_of_tiles_per_obsid(self):
        all_data = pd.read_parquet(self.dbname)
        return all_data.groupby("image_name").image_id.nunique()

    @property
    def EDRINDEX_meta_path(self):
        return self.savefolder / f"{self.catalog}_EDRINDEX_metadata.csv"

    def calc_metadata(self):
        if not self.EDRINDEX_meta_path.exists():
            NAs = p4meta.get_north_azimuths_from_SPICE(self.obsids)
            edrindex = get_index("mro.hirise", "edr")
            p4_edr = (
                edrindex[edrindex.OBSERVATION_ID.isin(self.obsids)]
                .query('CCD_NAME=="RED4"')
                .drop_duplicates(subset="OBSERVATION_ID")
            )
            p4_edr = p4_edr.set_index("OBSERVATION_ID").join(
                NAs.set_index("OBSERVATION_ID")
            )
            p4_edr = p4_edr.join(self.get_no_of_tiles_per_obsid())
            p4_edr.rename(dict(image_id="# of tiles"), axis=1, inplace=True)
            p4_edr["map_scale"] = 0.25 * p4_edr.BINNING
            p4_edr.reset_index(inplace=True)
            p4_edr.to_csv(self.EDRINDEX_meta_path)
        else:
            p4_edr = pd.read_csv(self.EDRINDEX_meta_path)
        cols = [
            "OBSERVATION_ID",
            "IMAGE_CENTER_LATITUDE",
            "IMAGE_CENTER_LONGITUDE",
            "SOLAR_LONGITUDE",
            "START_TIME",
            "map_scale",
            "north_azimuth",
            "# of tiles",
        ]
        metadata = p4_edr[cols]
        metadata.to_csv(self.metadata_path, index=False, float_format="%.7f")
        LOGGER.info("Wrote %s", str(self.metadata_path))

    def calc_tile_coordinates(self):
        cubepaths = [P4Mosaic(obsid).mosaic_path for obsid in self.obsids]

        todo = []
        for cubepath in cubepaths:
            tc = TileCalculator(cubepath, read_data=False, dbname=self.dbname)
            if not tc.campt_results_path.exists():
                todo.append(cubepath)

        def get_tile_coords(cubepath):
            from planet4.projection import TileCalculator##TODO is that import necessary

            tilecalc = TileCalculator(cubepath, dbname=self.dbname)
            tilecalc.calc_tile_coords()

        if not len(todo) == 0:
            _ = execute_in_parallel(get_tile_coords, todo)##Execute in parallel?

        bucket = []
        for cubepath in tqdm(cubepaths):
            tc = TileCalculator(cubepath, read_data=False, dbname=self.dbname)
            bucket.append(tc.tile_coords_df)
        coords = pd.concat(bucket, ignore_index=True, sort=False)
        coords.to_csv(self.tile_coords_path, index=False, float_format="%.7f")
        LOGGER.info("Wrote %s", str(self.tile_coords_path))

    @property
    def COLS_TO_MERGE(self):
        return [
            "obsid",
            "image_x",
            "image_y",
            "BodyFixedCoordinateX",
            "BodyFixedCoordinateY",
            "BodyFixedCoordinateZ",
            "PlanetocentricLatitude",
            "PlanetographicLatitude",
            "PositiveEast360Longitude",
        ]

    def merge_fnotch_results(self, fans, blotches):
        """Average multiple objects from fnotching into one.

        Because fnotching can compare the same object with more than one, it can appear more than once
        with different `vote_ratio` values in the results. We merge them here into one, simply
        averaging the vote_ratio. This increases the value of the `vote_ratio` number as it now
        has been created by several comparisons. It only occurs for 0.5 % of fans though.
        """
        out = []
        for df in [fans, blotches]:
            averaged = df.groupby("marking_id").mean(numeric_only=True)
            tmp = df.drop_duplicates(subset="marking_id").set_index("marking_id")
            averaged = averaged.join(tmp[["image_id", "obsid"]], how="inner")
            out.append(averaged.reset_index())

        return out

    def merge_all(self):
        # read in data files
        fans = pd.read_csv(self.fan_file)
        blotches = pd.read_csv(self.blotch_file)
        meta = pd.read_csv(self.metadata_path, dtype="str")
        tile_coords = pd.read_csv(self.tile_coords_path, dtype="str")

        # average multiple fnotch results
        fans, blotches = self.merge_fnotch_results(fans, blotches)

        # merge meta
        cols_to_merge = [
            "OBSERVATION_ID",
            "SOLAR_LONGITUDE",
            "north_azimuth",
            "map_scale",
        ]
        fans = fans.merge(
            meta[cols_to_merge], left_on="obsid", right_on="OBSERVATION_ID"
        )
        blotches = blotches.merge(
            meta[cols_to_merge], left_on="obsid", right_on="OBSERVATION_ID"
        )

        # drop unnecessary columns
        tile_coords.drop(
            self.DROP_FOR_TILE_COORDS, axis=1, inplace=True, errors="ignore"
        )
        # save cleaned tile_coords
        tile_coords.rename({"image_id": "tile_id"}, axis=1, inplace=True)
        tile_coords.to_csv(
            self.tile_coords_path_final, index=False, float_format="%.7f"
        )

        # merge campt results into catalog files
        fans, blotches = self.merge_campt_results(fans, blotches)

        # write out fans catalog
        fans.vote_ratio.fillna(1, inplace=True)
        fans.version = fans.version.astype("int")
        fans.rename(
            {
                "image_id": "tile_id",
                "SOLAR_LONGITUDE": "l_s",
                "PositiveEast360Longitude": "Longitude",
            },
            axis=1,
            inplace=True,
        )
        fans[self.FAN_COLUMNS_AS_PUBLISHED].to_csv(self.fan_merged, index=False)
        LOGGER.info("Wrote %s", str(self.fan_merged))

        # write out blotches catalog
        blotches.vote_ratio.fillna(1, inplace=True)
        blotches.rename(
            {
                "image_id": "tile_id",
                "SOLAR_LONGITUDE": "l_s",
                "PositiveEast360Longitude": "Longitude",
            },
            axis=1,
            inplace=True,
        )
        blotches[self.BLOTCH_COLUMNS_AS_PUBLISHED].to_csv(
            self.blotch_merged, index=False
        )
        LOGGER.info("Wrote %s", str(self.blotch_merged))

    def calc_marking_coordinates(self):
        fans = pd.read_csv(self.fan_file)
        blotches = pd.read_csv(self.blotch_file)
        combined = pd.concat([fans, blotches], sort=False)

        for obsid in tqdm(self.obsids):
            data = combined[combined.image_name == obsid]
            xy = XY2LATLON(data, self.savefolder, overwrite=self.overwrite)
            xy.process_inpath()

    def collect_marking_coordinates(self):
        bucket = []
        for obsid in self.obsids:
            xy = XY2LATLON(None, self.savefolder, obsid=obsid)
            bucket.append(pd.read_csv(xy.savepath).assign(obsid=obsid))

        ground = pd.concat(bucket, sort=False).drop_duplicates()
        ground.rename(dict(Sample="image_x", Line="image_y"), axis=1, inplace=True)
        return ground

    def fix_marking_coordinates_precision(self, df):
        fname = "tempfile.csv"
        df.to_csv(fname, float_format="%.7f")
        return pd.read_csv(fname, dtype="str")

    def merge_campt_results(self, fans, blotches):
        INDEX = ["obsid", "image_x", "image_y"]

        ground = self.collect_marking_coordinates().round(decimals=7)
        # ground = self.fix_marking_coordinates_precision(ground)
        fans = fans.merge(ground[self.COLS_TO_MERGE], on=INDEX)
        blotches = blotches.merge(ground[self.COLS_TO_MERGE], on=INDEX)
        return fans, blotches

    def perform_clustering(self):
        lazy_results = []

    def launch_catalog_production(self):
        # check for data that is unprocessed
        self.check_for_todo()

        # perform the clustering
        if len(self.todo) > 0:
            LOGGER.info("Performing the clustering.")
            results = cluster_obsid_parallel(self.todo, self.catalog, self.dbname)

            # create marking_ids
            fan_id = fan_id_generator()
            blotch_id = blotch_id_generator()
            for obsid in self.todo:
                paths = get_L1A_paths(obsid, self.catalog)
                for path in paths:
                    add_marking_ids(path, fan_id, blotch_id)

            # fnotch and apply cuts
            LOGGER.info("Start fnotching")
            results = fnotch_obsid_parallel(self.todo, self.catalog)

        # create summary CSV files of the clustering output
        LOGGER.info("Creating L1C fan and blotch database files.")
        create_roi_file(self.obsids, self.catalog, self.catalog)

        LOGGER.info("Creating the required RED45 mosaics for ground projections.")
        results = execute_in_parallel(create_RED45_mosaic, self.obsids)

        LOGGER.info("Calculating the center ground coordinates for all P4 tiles.")
        self.calc_tile_coordinates()

        LOGGER.info("Calculating ground coordinates for catalog.")
        self.calc_marking_coordinates()

        # calculate all metadata required for P4 analysis
        LOGGER.info("Writing summary metadata file.")
        self.calc_metadata()
        # merging metadata
        self.merge_all()

In [ ]:
#| export 
def fan_id_generator():
    for newid in itertools.product(string.digits + "abcdef", repeat=6):
        yield "F" + "".join(newid)


def blotch_id_generator():
    for newid in itertools.product(string.digits + "abcdef", repeat=6):
        yield "B" + "".join(newid)